In [2]:
from dotenv import load_dotenv
import sys

sys.path.append("../common")
load_dotenv()

True

In [3]:
import os
from langsmith_tracker import set_tracking
from langchain_print import stream_response
from multimodal import MultiModal
from tools_news import GoogleNews

# 인스턴스를 생성할 때 필요한 매개변수를 전달합니다.
set_tracking(project_name="15.Bind-Tools")

Langsmith 추적이 활성화되었습니다. [프로젝트명: 15.Bind-Tools]


### LLM 에 도구 바인딩(Binding Tools)

LLM 에 받구를 바인딩하는 기법입니다.  
LLM 에 도구를 바인드하여 스스로 판단하여 상황들에 따라 도구들을 적절히 사용할 수 있게 합니다.  
**Function Calling**, **Tool Calling** 과 같이 표현하며 목적으로는 LLM 에 도구를 바인드하는 행위입니다.  
  
도구 호출을 지원하는 LangChain Chat Model 은 `.bind_tools()` 메서드를 구현하여 Langchain 도구 객체, Pydantic 클래스 또는 JSON 스키마 목록을 수신하고 공급자별 예상 형식으로 채팅 모델에 바인딩 합니다.

### LLM에 바인딩할 Tool 정의

LLM 에 tools 를 한번에 여래개를 선언후 바인딩 할 수 있습니다. tools 를 선언하는 예시입니다.

In [26]:
import re
import requests
from bs4 import BeautifulSoup
from langchain.agents import tool


# 도구 정의
@tool
def get_word_length(word: str) -> int:
    """Return the length of a word."""
    return len(word)


@tool
def add_function(a: float, b: float) -> float:
    """Add two numbers together"""
    return a + b


@tool
def naver_news_crawl(news_url: str) -> str:
    """Crawls a NAVER(www.naver.com) news atricle and returns the body content"""
    # HTTP GET 요청
    response = requests.get(news_url)

    # 요청 확인
    if response.status_code == 200:
        # BeautifulSoup을 사용하여 HTML 파싱
        soup = BeautifulSoup(response.text, "html.parser")

        # 원하는 정보 추출
        title = soup.find("h2", id="title_area").get_text()
        content = soup.find("div", id="contents").get_text()
        cleaned_title = re.sub(r"\n{2,}", "\n", title)
        cleaned_content = re.sub(r"\n{2,}", "\n", content)
    else:
        print(f"HTTP 요청 실패, 응답 코드 : {response.status_code}")

    return f"{cleaned_title}\n{cleaned_content}"


tools = [get_word_length, add_function, naver_news_crawl]

#### `bind_toos()` 로 LLM 에 바인딩

In [7]:
from langchain_openai import ChatOpenAI

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# 도구 바인딩
llm_with_tools = llm.bind_tools(tools)

도구 바인딩이 되어 있으면 바로 원하는 대답을 볼 수 없습니다. 도구를 사용했는지 안했는지는 `.tool_calls` 을 호출해볼면 알 수 있습니다.  
type : `tool_call` 이 있으면 도구를 사용한 것입니다. 도구를 사용한 제대로된 답변을 받기 위해서는 직접 도구 호출을 해줘야 합니다.  
실행 결과
- name : 도구의 이름을 의미합니다.
- args : 도구에 전달되는 인자를 의미합니다.

In [8]:
llm_with_tools.invoke("What is the length of the word 'Bitcoin price rises'").tool_calls

[{'name': 'get_word_length',
  'args': {'word': 'Bitcoin price rises'},
  'id': 'call_9RpHrDdwmvWTshEBfaC51aUn',
  'type': 'tool_call'}]

#### `JsonOutputToosParser` 를 연결하여 `tool_calls` 를 parsing 한 결과

`JsonOutputToosParser` 로 `tool_calls` 의 결과를 파싱하면 인자와 실행해야 할 함수 정보를 알 수 있습니다.

In [9]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# 도구 바인딩 + 도구 파서
chain = llm_with_tools | JsonOutputToolsParser(tools=tools)

# 실행 결과
tool_call_results = chain.invoke("What is the length of the word 'Bitcoin price rises'")

In [10]:
print(tool_call_results)

[{'args': {'word': 'Bitcoin price rises'}, 'type': 'get_word_length'}]


In [11]:
print(tool_call_results, end="\n\n==========\n\n")
# 첫 번째 도구 호출 결과
single_result = tool_call_results[0]
# 도구 이름
print(single_result["type"])
# 도구 인자
print(single_result["args"])

[{'args': {'word': 'Bitcoin price rises'}, 'type': 'get_word_length'}]


get_word_length
{'word': 'Bitcoin price rises'}


In [12]:
tool_call_results[0]["type"], tools[0].name

('get_word_length', 'get_word_length')

그래서 도구를 직접 호출하고 실행할 수 있는 함수를 정의하고 실행합니다.

In [14]:
def execute_tool_calls(tool_call_results):
    """
    도구 호출 결과를 실행하는 함수

    :param tool_call_results: 도구 호출 결과 리스트
    :param tools: 사용 가능한 도구 리스트
    """
    # 도구 호출 결과 리스트를 순회합니다.
    for tool_call_result in tool_call_results:
        # 도구의 이름과 인자를 추출합니다.
        tool_name = tool_call_result["type"]  # 도구의 이름(함수명)
        tool_args = tool_call_result["args"]  # 도구에 전달되는 인자

        # 도구 이름과 일치하는 도구를 찾아 실행합니다.
        # next() 함수를 사용하여 일치하는 첫 번째 도구를 찾습니다.
        matching_tool = next((tool for tool in tools if tool.name == tool_name), None)

        if matching_tool:
            # 일치하는 도구를 찾았다면 해당 도구를 실행합니다.
            result = matching_tool.invoke(tool_args)
            # 실행 결과를 출력합니다.
            print(
                f"[실행도구] {tool_name}\n[Argument] {tool_args}\n[실행결과] {result}"
            )
        else:
            # 일치하는 도구를 찾지 못했다면 경고 메시지를 출력합니다.
            print(f"경고: {tool_name}에 해당하는 도구를 찾을 수 없습니다.")


# 도구 호출 실행
# 이전에 얻은 tool_call_results를 인자로 전달하여 함수를 실행합니다.
execute_tool_calls(tool_call_results)

[실행도구] get_word_length
[Argument] {'word': 'Bitcoin price rises'}
[실행결과] 19


#### bind_tools + Parser + Execution
llm_with_tools : 도구를 바인딩한 모델  
JsonOutputToolsParser : 도구 호출 결과를 파싱하는 파서  
execute_tool_calls : 도구 호출 결과를 실행하는 함수  
  

흐름 정리
1. 모델에 도구를 바인딩
2. 도구 호출 결과를 파싱
3. 도구 호출 결과를 실행

In [17]:
from langchain_core.output_parsers.openai_tools import JsonOutputToolsParser

# bind_tools + Parser + Execution
chain = llm_with_tools | JsonOutputToolsParser(tools=tools) | execute_tool_calls

In [18]:
# 실행 결과
chain.invoke("What is the length of the word 'Bitcoin price rises'")

[실행도구] get_word_length
[Argument] {'word': 'Bitcoin price rises'}
[실행결과] 19


In [19]:
# 실행 결과
chain.invoke("188.7 + 170.2")

[실행도구] add_fucntion
[Argument] {'a': 188.7, 'b': 170.2}
[실행결과] 358.9


In [24]:
# 실행 결과
chain.invoke(
    "뉴스 기사를 크롤링해주세요 : https://n.news.naver.com/mnews/article/001/0015077231"
)

[실행도구] naver_news_crawl
[Argument] {'news_url': 'https://n.news.naver.com/mnews/article/001/0015077231'}
[실행결과] 투자 스팸문자 사기 막는다…대량 발송·수신 차단 이달 시작

투자 유도 스팸 급증에 금감원·KISA·이통3사 예방책 마련 
금융감독원[연합뉴스TV 제공]    (서울=연합뉴스) 임수정 기자 = 금융감독원은 한국인터넷진흥원(KISA), 이동통신 3사와 함께 투자 스팸 문자 차단 방안을 마련해 이달 시행한다고 1일 밝혔다.    불법 개연성이 높은 키워드가 포함된 투자 유인 스팸 문자를 사전에 차단하는 방안들이다.    올해 상반기 투자 유인 불법 스팸 신고는 6천67만건으로 작년 하반기(673만건) 대비 약 8배 급증하는 등 투자 사기 피해를 예방할 필요성이 커졌다.    우선 KISA의 '스팸 전화번호 블랙리스트 제도' 적용 범위를 투자 유인 스팸 문자로 확대 적용한다.    금감원과 KISA는 투자 유인 스팸 문자 약 2만여개를 분석해 블랙리스트에 활용할 불법 금융투자 키워드를 선정했다.    이동통신사들은 '필터링 서비스'에 투자 관련 필터링을 강화하기로 했다.    현재 이동통신사들은 각 회사의 고유 분석 로직 및 차단 시스템을 통해 스팸을 걸러내고 있는데, 금감원과 KISA가 분석한 키워드를 추가 반영하기로 했다.    이들 3사의 수신 필터링 서비스는 해외에서 발송하는 스팸 문자도 걸러낼 수 있다는 장점이 있다.    금감원은 "스팸 문자 발송과 수신 차단 방안이 상호 보완적으로 스팸 문자를 차단할 수 있을 것"이라고 기대했다.    금감원은 불법 금융투자 스팸 문자 발송·수신 차단 방안을 테스트 해 본 결과 투자 유인 유형의 스팸 문자를 20%가량 추가 차단하는 효과가 있었다고 설명했다.    sj9974@yna.co.kr
임수정(sj9974@yna.co.kr)
기자 프로필
연합뉴스
연합뉴스
임수정 기자
구독
구독중
구독자 0
응원수
0
저축은행 절반은

### bind_tools > Agnet & AgentExecutor 로 대체

`bind_tools()` 는 모델에 사용할 수 있는 스키마(도구)를 제공합니다.  
`AgentExecutor` 는 실제로 llm 호출, 올바른 도구로 라우팅, 실행, 모델 재호출 등을 위한 실행 루프를 생성합니다.  

In [28]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_openai import ChatOpenAI

# Agent 프롬프트 생성
prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are very powerful assistant, but don't know current events",
        ),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

# 모델 생성
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

In [29]:
from langchain.agents import create_tool_calling_agent
from langchain.agents import AgentExecutor

# 이전에 정의한 도구 사용
tools = [get_word_length, add_function, naver_news_crawl]

# Agent 생성
agent = create_tool_calling_agent(llm, tools, prompt)

# AgentExecutor 생성
agent_executor = AgentExecutor(
    agent=agent,
    tools=tools,
    verbose=True,
    handle_parsing_errors=True,
)

In [30]:
# Agent 실행
result = agent_executor.invoke(
    {"input": "How many letters in the word `Bitcoin price rises`?"}
)

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `get_word_length` with `{'word': 'Bitcoin price rises'}`


19The phrase "Bitcoin price rises" contains 19 letters.

> Finished chain.
The phrase "Bitcoin price rises" contains 19 letters.


In [31]:
# Agent 실행
result = agent_executor.invoke({"input": "188.7 + 170.2 의 계산 결과는?"})

# 결과 확인
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 188.7, 'b': 170.2}`


358.9188.7 + 170.2의 계산 결과는 358.9입니다.

> Finished chain.
188.7 + 170.2의 계산 결과는 358.9입니다.


In [35]:
# Agent 실행
result = agent_executor.invoke({"input": "188.7 + 170.2 + 208.7 + 40.2의 계산 결과는?"})

# 결과 확인
print(result["output"])
print("==========\n")
print(188.7 + 170.2 + 208.7 + 40.2)



> Entering new AgentExecutor chain...

Invoking: `add_function` with `{'a': 188.7, 'b': 170.2}`


358.9
Invoking: `add_function` with `{'a': 208.7, 'b': 40.2}`


248.89999999999998
Invoking: `add_function` with `{'a': 358.9, 'b': 248.89999999999998}`


607.8188.7 + 170.2 + 208.7 + 40.2의 계산 결과는 607.8입니다.

> Finished chain.
188.7 + 170.2 + 208.7 + 40.2의 계산 결과는 607.8입니다.

607.8


In [33]:
result = agent_executor.invoke(
    {
        "input": "뉴스 기사를 크롤링해주세요 : https://n.news.naver.com/mnews/article/001/0015077231"
    }
)
print(result["output"])



> Entering new AgentExecutor chain...

Invoking: `naver_news_crawl` with `{'news_url': 'https://n.news.naver.com/mnews/article/001/0015077231'}`


투자 스팸문자 사기 막는다…대량 발송·수신 차단 이달 시작

투자 유도 스팸 급증에 금감원·KISA·이통3사 예방책 마련 
금융감독원[연합뉴스TV 제공]    (서울=연합뉴스) 임수정 기자 = 금융감독원은 한국인터넷진흥원(KISA), 이동통신 3사와 함께 투자 스팸 문자 차단 방안을 마련해 이달 시행한다고 1일 밝혔다.    불법 개연성이 높은 키워드가 포함된 투자 유인 스팸 문자를 사전에 차단하는 방안들이다.    올해 상반기 투자 유인 불법 스팸 신고는 6천67만건으로 작년 하반기(673만건) 대비 약 8배 급증하는 등 투자 사기 피해를 예방할 필요성이 커졌다.    우선 KISA의 '스팸 전화번호 블랙리스트 제도' 적용 범위를 투자 유인 스팸 문자로 확대 적용한다.    금감원과 KISA는 투자 유인 스팸 문자 약 2만여개를 분석해 블랙리스트에 활용할 불법 금융투자 키워드를 선정했다.    이동통신사들은 '필터링 서비스'에 투자 관련 필터링을 강화하기로 했다.    현재 이동통신사들은 각 회사의 고유 분석 로직 및 차단 시스템을 통해 스팸을 걸러내고 있는데, 금감원과 KISA가 분석한 키워드를 추가 반영하기로 했다.    이들 3사의 수신 필터링 서비스는 해외에서 발송하는 스팸 문자도 걸러낼 수 있다는 장점이 있다.    금감원은 "스팸 문자 발송과 수신 차단 방안이 상호 보완적으로 스팸 문자를 차단할 수 있을 것"이라고 기대했다.    금감원은 불법 금융투자 스팸 문자 발송·수신 차단 방안을 테스트 해 본 결과 투자 유인 유형의 스팸 문자를 20%가량 추가 차단하는 효과가 있었다고 설명했다.    sj9974@yna.co.kr
임수정(sj9974@yna.co.kr)
기자 프로필
연합뉴스
연합